In [1]:
import carla
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal
import learn2learn as l2l
import csv
import time

import torch.nn as nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(NeuralNetwork, self).__init__()

        self.l1 = nn.Linear(state_dim, 64)
        self.l2 = nn.Linear(64, 64)
        self.l3 = nn.Linear(64, action_dim)

    def forward(self, state):
        x = F.relu(self.l1(state))
        x = F.relu(self.l2(x))
        #action_logits = self.l3(x)
        actions = self.l3(x)
        #ction_probs = F.softmax(action_logits, dim=1)

        return actions


class CarlaEnvironment:
    def __init__(self, world, vehicle_blueprint, waypoints):
        self.world = world
        self.vehicle_blueprint = vehicle_blueprint
        self.waypoints = waypoints
        self.actorlist = []
        self.current_waypoint_list = []
        
        # self.vehicle = self.world.spawn_actor(self.vehicle_blueprint, carla.Transform(carla.Location(x=-23.6,y=137.5,z=1),carla.Rotation(yaw=0)))
        
    def reset(self):
        self.vehicle = self.world.spawn_actor(self.vehicle_blueprint, carla.Transform(carla.Location(x=-23.6,y=137.5,z=1),carla.Rotation(yaw=0)))
        self.current_waypoint = 0
        self.current_waypoint_list.append(self.current_waypoint)
        self.moving = False
        return self.get_state()

    def step(self, action):
        action = action.detach().numpy()
        print("Waypoint number: ", self.current_waypoint)
        self.current_waypoint_list.append(self.current_waypoint)
        #print(action)
        # Apply actions: Throttle, Steering, Brake
        if self.current_waypoint <10:
            action = [1.0, 0.0, 0.0]
        if self.current_waypoint ==10:
            self.moving = True            
        self.vehicle.apply_control(carla.VehicleControl(throttle = float(action[0]) , steer = float(action[1]) , brake = float(action[2])))
       
        #self.actorlist.append(self.vehicle)

        # Get next state
        next_state = self.get_state()

        # Calculate reward
        reward = self.calculate_reward(next_state)
       
        # Check if the episode is done
        done = self.is_done(next_state)

        return next_state, reward, done

#     def get_state(self):
#         curr_vel = self.vehicle.get_velocity()
#         curr_yaw = self.vehicle.get_transform().rotation.yaw
#         waypoint = self.waypoints[self.current_waypoint]
#         return np.array([curr_vel, curr_yaw - waypoint[1], curr_yaw - waypoint[2], waypoint[3]])
    
    def get_state(self):
        state = []
        curr_yaw = self.vehicle.get_transform().rotation.yaw
        
        
        for i in range(10):
            if self.current_waypoint + i < len(self.waypoints):
                waypoint = self.waypoints[self.current_waypoint + i]
                curr_vel = self.vehicle.get_velocity()
                curr_loc = self.vehicle.get_location()
                vx = curr_vel.x
                vy = curr_vel.y
                v = ((vx)**2+(vy)**2)**0.5
                if curr_loc.x==0 and curr_loc.y==0:
                    curr_loc = carla.Location(x=waypoint[4], y=waypoint[5])
                distance = curr_loc.distance(carla.Location(x=waypoint[4],y=waypoint[5]))
                
                state.extend([v, curr_yaw - waypoint[1], curr_yaw - waypoint[2], distance ])
            else:
                state.extend([0, 0, 0, 0])
               
        return np.array(state)

    def generate_waypoints(self):
        # Implement waypoint generation logic
        pass

    def calculate_reward(self, state):
        reward = -(np.linalg.norm(state[0] - self.waypoints[self.current_waypoint][0]) - (np.linalg.norm (state[1])) - (np.linalg.norm ( state[2])) - (np.linalg.norm (state[3])) + self.current_waypoint)
        return reward

    def is_done(self, state):
        curr_loc = self.vehicle.get_location()
        if curr_loc.x==0 and curr_loc.y==0:
            curr_loc = carla.Location(x=self.waypoints[self.current_waypoint][4],y=self.waypoints[self.current_waypoint][5])
 
        distance = curr_loc.distance(carla.Location(x=self.waypoints[self.current_waypoint][4],y=self.waypoints[self.current_waypoint][5]))

        if distance < 1:  # If the vehicle is close to the next waypoint
            self.current_waypoint += 1

        if self.current_waypoint >= len(self.waypoints):
            return True
    
        
        return False

    def set_weather(self, weather):
        self.world.set_weather(weather)
        
    def destroy(self):
        for actor in self.actorlist:
            actor.destroy()
        print("All Cleared")


def train_ppo_maml(env, state_dim, action_dim, lr=1e-4, n_iters=1000000, inner_loop_steps=5):
    base_model = NeuralNetwork(state_dim, action_dim)
    maml = l2l.algorithms.MAML(base_model, lr=lr)
    optimizer = optim.Adam(maml.parameters(), lr=lr)
    weather = carla.WeatherParameters.HardRainNoon
        #for weather in weather_conditions:
    env.set_weather(weather)
    for iteration in range(n_iters):
        
        trajectories, state_tensors, action_tensors = sample_trajectories(env, maml)
        state_tensors_stacked = torch.stack(state_tensors)
        action_tensors_stacked = torch.stack(action_tensors)
        for _ in range(inner_loop_steps):
            clone = maml.clone()
            error = nn.MSELoss()(clone(state_tensors_stacked), action_tensors_stacked)
            clone.adapt(error)
            error = nn.MSELoss()(clone(state_tensors_stacked), action_tensors_stacked)
            error.backward(retain_graph=True)
        optimizer.zero_grad()
        optimizer.step()

        if iteration % 1000 == 0:
            print("Iteration: ", iteration)


def sample_weather_conditions(num_weather_conditions):
    return carla.WeatherParameters.HardRainNoon

def sample_trajectories(env, model, num_episodes=10, max_episode_length=1000):
#     model.to("cpu")
    trajectories = []
    traj_states = []
    traj_action = []
    state_tensors = []
    action_tensors = []

    for _ in range(num_episodes):
        
        state = env.reset()              
        trajectory = []
        print("trajectories: ",len(trajectories))
        while True:
            
#             state_tensor = torch.tensor(state).to(torch.float32)
            state_tensor = torch.from_numpy(state.astype(np.float32))
            state_tensors.append(state_tensor)
            action_tensor = model(state_tensor)
#             action_distribution = Normal(action_mean, torch.tensor(0.1))
#             action = action_distribution.sample().detach().numpy().astype(float)
            
            action = action_tensor
#             action_tensor = torch.tensor(action, dtype=torch.float32)
            action_tensors.append(action)

            next_state, reward, done = env.step(action)
            trajectory.append((state, action, reward, next_state, done))

            state = next_state
            waypoint_copy = env.current_waypoint
            # if not j%200 : print("Reward: ", reward, )
            time.sleep(0.1)
            if len(env.current_waypoint_list) >=21:
                if env.current_waypoint_list[len(env.current_waypoint_list)-1] == env.current_waypoint_list[len(env.current_waypoint_list) - 20] and env.moving:
                    done = True
            #state_tensors = torch.tensor(state_tensor, dtype=torch.float32)
            if done or (env.current_waypoint == len(env.waypoints) - 1):
                env.vehicle.destroy()
                break  

        trajectories.append(trajectory)
    return trajectories, state_tensors, action_tensors

            
def read_waypoints(file_path):
    waypoints = []
    with open(file_path, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            v, yaw_c, yaw_n, r, x, y = map(float, row)
            waypoints.append((v, yaw_c, yaw_n, r, x, y))
            
    return waypoints                        

In [2]:
if __name__ == '__main__':

    client = carla.Client('localhost', 2000)
    client.set_timeout(2.0)

    world = client.get_world()
    blueprint_library = world.get_blueprint_library()
    vehicle_blueprint = blueprint_library.filter("wrangler_rubicon")[0]

    waypoints = read_waypoints("waypoint_with_xy.csv")
    env = CarlaEnvironment(world, vehicle_blueprint, waypoints)  # Implement CarlaEnvironment with the required state and action spaces

    state_dim = 40
    action_dim = 3

    train_ppo_maml(env, state_dim, action_dim)

        

trajectories:  0
Waypoint number:  0
Waypoint number:  0
Waypoint number:  0
Waypoint number:  1
Waypoint number:  1
Waypoint number:  1
Waypoint number:  1
Waypoint number:  2
Waypoint number:  2
Waypoint number:  2
Waypoint number:  2
Waypoint number:  3
Waypoint number:  3
Waypoint number:  4
Waypoint number:  4
Waypoint number:  5
Waypoint number:  6
Waypoint number:  6
Waypoint number:  7
Waypoint number:  8
Waypoint number:  9
Waypoint number:  10
Waypoint number:  11
Waypoint number:  12
Waypoint number:  13
Waypoint number:  14
Waypoint number:  15
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
Waypoint number:  16
trajectories:  1


In the example code you provided, X represents the input features (or states in the case of a reinforcement learning problem), and Y represents the target values (or true action values in a reinforcement learning problem) for training the model.

The code snippet demonstrates how to use Learn2Learn's MAML implementation for a simple linear regression problem with 20 input features and 10 output targets. Here's a brief explanation of each line of code:

    linear = l2l.algorithms.MAML(nn.Linear(20, 10), lr=0.01): Create a MAML instance with a simple linear model and a learning rate of 0.01.
    clone = linear.clone(): Clone the base model to create a temporary model for adaptation.
    error = loss(clone(X), y): Compute the loss (error) between the model's predictions on the input X and the true target values Y.
    clone.adapt(error): Update the temporary model's parameters using the computed loss (one step of gradient descent).
    error = loss(clone(X), y): Compute the loss again after adaptation.
    error.backward(): Perform backpropagation to compute gradients for updating the base model.

In the context of your problem, X would correspond to the state values (with 40 elements, considering the current waypoint and the upcoming 9 waypoints), and Y would be the true action values to follow the given waypoints closely.

Keep in mind that this example is for a simple regression problem, and you would need to adapt it to suit the reinforcement learning setting, using the appropriate loss function and input-output pairs (states and actions) from the sampled trajectories.